In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a8d642d31fa1dc6f91c29515484fb48d45ce11e2a63aab8d4e5eed93b4f968aa
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install requests

In [13]:
import os
import sys
import re
import json
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [14]:
# Initialize a Spark session
spark = SparkSession.builder.appName("Aerodrome_Info_API").getOrCreate()

In [15]:
# Read VRA data from a Parquet file
df_vra = spark.read.parquet("/content/VRA")


In [16]:
df_vra.show(truncate=False)

+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|chegada_prevista   |chegada_real       |codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icao_aerodromo_origem|icao_empresa_aerea|numero_voo|partida_prevista   |partida_real       |situacao_voo|
+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:30:00|0                 |N/A                 |X                |KORD                  |SBGR                 |UAL               |844       |2021-11-11 22:00:00|2021-11-11 22:14:00|REALIZADO   |
|2021-11-15 08:30:00|2021-11-15 08:30:00|0                 |N/A                 |X      

In [17]:
df_air_cia = spark.read.parquet("/content/AIR_CIA")

In [18]:
df_air_cia.show(truncate=False)

+------------------------------------------------------------------+----+----+------------------+------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+-----------------------------------+----------------------------+------------------------+--------------------+
|razao_social                                                      |icao|iata|cnpj              |atividades_aereas                                                                               |endereco_sede                                                                                                                                                       |telefone                                     |e_mail                             |decisao_operacional         |data_decisao_

##Scraping da API

In [12]:
# Define your RapidAPI key
# por segurança essa key deve ser ocultada e passada como variavel de ambiente
rapidapi_key = "a15526b664msh3495e669af65699p1cc753jsn486844326116"

In [13]:
# Define the API endpoint
api_url = "https://airport-info.p.rapidapi.com/airports"

In [14]:
# Create an empty DataFrame to store aerodrome information and rename it to df_aerodramo
df_aerodramo = spark.createDataFrame([], schema=df_vra.schema)

In [17]:
# Loop through ICAO codes in the df_air_cia DataFrame
for row in df_air_cia.rdd.collect():
    icao_code = row["icao"]

    # Define query parameters for the API
    query_params = {"iata": icao_code}

    # Set headers with your RapidAPI key and host
    headers = {
        "X-RapidAPI-Key": rapidapi_key,
        "X-RapidAPI-Host": "airport-info.p.rapidapi.com"
    }

    # Make the API request
    response = requests.get(api_url, params=query_params, headers=headers)

    if response.status_code == 200:
        aerodrome_info = response.json()
        # Convert the aerodrome_info to a PySpark Row and append it to the DataFrame
        aerodrome_info_row = spark.createDataFrame([aerodrome_info], schema=df_air_cia.schema)
        df_aerodramo = df_aerodramo.union(aerodrome_info_row)



In [18]:
# Show a preview of the df_aerodramo DataFrame
df_aerodramo.show()

+----------------+------------+------------------+--------------------+-----------------+----------------------+--------------------+------------------+----------+----------------+------------+------------+
|chegada_prevista|chegada_real|codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icaoaerodromo_origem|icao_empresa_aerea|numero_voo|partida_prevista|partida_real|situacao_voo|
+----------------+------------+------------------+--------------------+-----------------+----------------------+--------------------+------------------+----------+----------------+------------+------------+
+----------------+------------+------------------+--------------------+-----------------+----------------------+--------------------+------------------+----------+----------------+------------+------------+



-teste da api


Failed to retrieve data from the API. Status code: 404


In [28]:
import http.client
conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")
headers = {
    'x-rapidapi-key': "a15526b664msh3495e669af65699p1cc753jsn486844326116",
    'x-rapidapi-host': "airport-info.p.rapidapi.com"
    }
conn.request("GET", "/airport?icao=KJFK", headers=headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"id":3406,"iata":"JFK","icao":"KJFK","name":"John F. Kennedy International Airport","location":"New York City, New York, United States","street_number":"","street":"","city":"","county":"Queens County","state":"New York","country_iso":"US","country":"United States","postal_code":"11430","phone":"+1 718-244-4444","latitude":40.64131,"longitude":-73.77814,"uct":-240,"website":"https://www.jfkairport.com/"}



In [34]:
##################### rascunho requests que deu
import requests

# Replace with your RapidAPI key
RAPIDAPI_KEY = 'a15526b664msh3495e669af65699p1cc753jsn486844326116'
icao_code = 'KJFK'  # Replace with the desired ICAO code

# Set up the API endpoint and headers
url = f'https://airport-info.p.rapidapi.com/airport?icao={icao_code}'
headers = {
    'X-RapidAPI-Host': 'airport-info.p.rapidapi.com',
    'X-RapidAPI-Key': RAPIDAPI_KEY
}

# Make an HTTP GET request to the API
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
    # Now you can process the data or create a PySpark DataFrame as needed.
else:
    print(f"Failed to retrieve data from the API. Status code: {response.status_code}")

{'id': 3406, 'iata': 'JFK', 'icao': 'KJFK', 'name': 'John F. Kennedy International Airport', 'location': 'New York City, New York, United States', 'street_number': '', 'street': '', 'city': '', 'county': 'Queens County', 'state': 'New York', 'country_iso': 'US', 'country': 'United States', 'postal_code': '11430', 'phone': '+1 718-244-4444', 'latitude': 40.64131, 'longitude': -73.77814, 'uct': -240, 'website': 'https://www.jfkairport.com/'}


#Teste dataframe com requests

In [ ]:
# Import the necessary libraries
from pyspark.sql import SparkSession
import requests

# Replace with your RapidAPI key
RAPIDAPI_KEY = 'a15526b664msh3495e669af65699p1cc753jsn486844326116'

# Initialize a Spark session
spark = SparkSession.builder.appName("AirportInfo").getOrCreate()

# Read your DataFrame from a Parquet file
df_vra = spark.read.parquet("/content/VRA/")

# Function to retrieve aerodrome data for a specific ICAO code
def get_aerodrome_data(icao_code):
    url = f'https://airport-info.p.rapidapi.com/airport?icao={icao_code}'
    headers = {
        'X-RapidAPI-Host': 'airport-info.p.rapidapi.com',
        'X-RapidAPI-Key': RAPIDAPI_KEY
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for ICAO: {icao_code}")
        return None

# Iterate through distinct ICAO codes and retrieve aerodrome data
aerodrome_data = []
distinct_icao_codes = df_vra.select("icao_aerodromo_origem").distinct().rdd.flatMap(lambda x: x).collect()

for icao_code in distinct_icao_codes:
    data = get_aerodrome_data(icao_code)
    if data:
        aerodrome_data.append(data)

# Create a PySpark DataFrame from the obtained aerodrome data
aerodrome_df = spark.createDataFrame(aerodrome_data)

# Show the aerodrome data DataFrame
aerodrome_df.show()

# Stop the Spark session
#spark.stop()--------> executar novament depois


In [21]:
#a15526b664msh3495e669af65699p1cc753jsn486844326116

In [22]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Create a Spark session
spark = SparkSession.builder.appName("AirportInfo").getOrCreate()

# API URL
url = "https://airport-info.p.rapidapi.com/airport"

# Request headers with your API key
headers = {
    "X-RapidAPI-Host": "airport-info.p.rapidapi.com",
    "X-RapidAPI-Key": "a15526b664msh3495e669af65699p1cc753jsn486844326116"
}

# Make a GET request to the API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response into a list of dictionaries
    json_data = [response.json()]

    # Define the schema for the DataFrame
    schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("iata", StringType(), True),
        StructField("icao", StringType(), True),
        StructField("name", StringType(), True),
        StructField("location", StringType(), True),
        StructField("street_number", StringType(), True),
        StructField("street", StringType(), True),
        StructField("city", StringType(), True),
        StructField("county", StringType(), True),
        StructField("state", StringType(), True),
        StructField("country_iso", StringType(), True),
        StructField("country", StringType(), True),
        StructField("postal_code", StringType(), True),
        StructField("phone", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("uct", IntegerType(), True),
        StructField("website", StringType(), True)
    ])

    # Create the DataFrame from the list of dictionaries
    df = spark.createDataFrame(json_data, schema=schema)

    # Display the DataFrame
    df.show()

    # Optionally, you can perform further DataFrame operations, e.g., selecting specific columns
    df.select("iata", "icao", "name", "location", "country").show()

    # Stop the Spark session
    spark.stop()
else:
    print(f"API request failed with status code {response.status_code}")

# Be sure to replace "YOUR_RAPIDAPI_KEY_HERE" with your actual RapidAPI key.


+----+----+----+----+--------+-------------+------+----+------+-----+-----------+-------+-----------+-----+--------+---------+----+-------+
|  id|iata|icao|name|location|street_number|street|city|county|state|country_iso|country|postal_code|phone|latitude|longitude| uct|website|
+----+----+----+----+--------+-------------+------+----+------+-----+-----------+-------+-----------+-----+--------+---------+----+-------+
|NULL|NULL|NULL|NULL|    NULL|         NULL|  NULL|NULL|  NULL| NULL|       NULL|   NULL|       NULL| NULL|    NULL|     NULL|NULL|   NULL|
+----+----+----+----+--------+-------------+------+----+------+-----+-----------+-------+-----------+-----+--------+---------+----+-------+

+----+----+----+--------+-------+
|iata|icao|name|location|country|
+----+----+----+--------+-------+
|NULL|NULL|NULL|    NULL|   NULL|
+----+----+----+--------+-------+

